Qui sto cercando di capire se funziona con normalize=True. Provo con batch size 10**6 perche 1**5 era brutto

In [2]:
import sys
sys.path.append('..')

import utils, selex_dca, indep_sites
import adabmDCA
import selex_distribution, energy_models, tree, data_loading, training, callback, sampling

import torch
from utils import one_hot
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/home/scrotti/Aptamer2025py/selex_dca.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [15]:
experiment_id = "Dop8V030"
# round_ids = ["ARN", "R01", "R01CS", "R02N", "R02F", "R02S", "R02SF"]
round_ids = ["ARN", "R01", "R01CS", "R02N"]

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
dtype = torch.float32

In [16]:
sequences = []
for round_id in round_ids:
    s = utils.sequences_from_file(experiment_id, round_id, device)
    sequences.append(s)
    print(f"finished {round_id}")

finished ARN
finished R01
finished R01CS
finished R02N


In [17]:
sequences_oh = [one_hot(seq).to(dtype=dtype, device=device) for seq in sequences]

In [18]:
# del sequences

In [60]:
total_reads = torch.Tensor([s.shape[0] for s in sequences_oh]).to(device)
fi0, _, _ = utils.frequences_from_sequences_oh(sequences_oh[0])

In [163]:
tr = tree.Tree()
tr.add_node(-1, name = "R01")
tr.add_node(-1, name = "R01CS")
tr.add_node(0, name = "R02N")
# tr.add_node(0, name = "R02F")
# tr.add_node(0, name = "R02S")
# tr.add_node(0, name = "R02SF")

mode_names = ["target", "bead"]

selected_modes = torch.BoolTensor(
    [[1, 1],[0, 1], [1, 1]]
).to(device)

In [164]:
L, q = sequences_oh[0][0].shape

k = torch.zeros(L, q, dtype=dtype, device=device)
k = torch.log(fi0)
h_target = torch.randn(L, q, dtype=dtype, device=device)
J_target = torch.zeros(L, q, L, q, dtype=dtype, device=device)
h_bead = torch.randn(L, q, dtype=dtype, device=device)
J_bead = torch.zeros(L, q, L, q, dtype=dtype, device=device)

Ns0 = energy_models.IndepSites(k)
potts_target = energy_models.Potts(J_target, h_target)
potts_bead = energy_models.Potts(J_bead, h_bead)

ps = selex_distribution.MultiModeDistribution(potts_target, potts_bead, normalized=True)
model = selex_distribution.MultiRoundDistribution(Ns0, ps, tr, selected_modes)

In [165]:
batch_size = 10**6
data_loaders = [data_loading.SelexRoundDataLoader(seq_oh, batch_size=batch_size) for seq_oh in sequences_oh]
n_rounds = len(data_loaders) 

In [166]:
n_chains = 10**4

chains = training.init_chains(n_rounds, n_chains, L, q, device, dtype)
log_weights = torch.zeros(n_rounds, n_chains, device=device, dtype=dtype)

In [167]:
callbacks = [callback.ConvergenceMetricsCallback(), callback.PearsonCovarianceCallback()]

In [ ]:
n_sweeps = 10
lr = 0.01
target_pearson = 1
max_epochs = 200

%lprun -f training.train training.train(model, data_loaders, total_reads, chains, n_sweeps, lr, max_epochs, target_pearson, callbacks=callbacks)

 0.00%[                              ] Epoch: 0/200 [00:00, ?it/s]

In [ ]:
callbacks[0].plot();

In [ ]:
fig, ax = callbacks[1].plot()
ax.axhline(1, color='r', linestyle='--')

In [ ]:
from IPython.display import display, Latex

potts_zerosum = potts_target.set_zerosum_gauge()

pl, ax = plt.subplots(figsize=(3,3))
F = selex_dca.get_contact_map(potts_zerosum.J.detach())
im = ax.imshow(F)
ax.set_xlabel("i"); ax.set_ylabel("i")
ax.set_title(experiment_id)
plt.colorbar(im)
plt.tight_layout()
# im.set_clim(-0.0015, 0.0030)
display(Latex("$F_{ij}=\\sqrt {\\sum_{ab}(J_{ij}^{ab})^2}$"))

In [ ]:
plt.scatter(potts_bead.h.cpu().detach().numpy(), potts_target.h.cpu().detach().numpy())